<a href="https://colab.research.google.com/github/LennyRBriones/huggingface/blob/main/my_NLP_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Getting the Data

In [1]:
%%capture
!pip install datasets transformers evaluate

To this code we going to use the dataset MRPC, being one of the 10th in the benchmaark GLUE. 

In [3]:
from datasets import load_dataset

ds = load_dataset("glue", "mrpc") #the subset that we are downloading

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Now expoloring the data we can see that there are 2 sentences & 1 output

In [5]:
example = ds["train"][200]
example

{'sentence1': '" Smallpox is not the only threat to the public \'s health , and vaccination is not the only tool for smallpox preparedness , " Strom said .',
 'sentence2': '" Smallpox is not the only threat to the nation \'s health , and vaccination is not the only tool for preparedness , " his introductory statement says .',
 'label': 1,
 'idx': 223}

now we have our labels to explore

In [8]:
ds["train"].features["label"]

ClassLabel(names=['not_equivalent', 'equivalent'], id=None)

In [10]:
labels = ds["train"].features["label"]

In [11]:
labels.int2str(0)

'not_equivalent'

In [12]:
labels.int2str(1)

'equivalent'